## Overview
In this notebook, we outline how to create a WML Deployable function as a frontend to the trained machine learning model so that we can provide the required logging for Watson Openscale.


In [1]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
# Define end point url from deployed SPSS model
spss_scoring_url='https://us-south.ml.cloud.ibm.com/v3/wml_instances/8654e5c8-9bda-4800-9b4d-31d69fa179c3/deployments/e4a73684-8150-4fcb-a2a0-6cbf229ba8e0/online'

In [3]:
# Specify the credentials for Watson Machine Learning and define wml client
#@hidden_cell

wml_credentials={
  "apikey": "*",
  "iam_apikey_description": "*",
  "iam_apikey_name": "*",
  "iam_role_crn": "*",
  "iam_serviceid_crn": "*",
  "instance_id": "*",
  "password": "*",
  "url": "*",
  "username": "*"
}

client = WatsonMachineLearningAPIClient(wml_credentials)


In [4]:
# define values to be sent to deploy SPSS model for testing 
array_of_values_to_be_scored=["F","S",1,38000.000000,"N",24.393333,23.560000,0.000000,206.080000,0,"CC","Budget","Intnl_discount",229.640000,3]
another_array_of_values_to_be_scored=["M","M",2,29616.000000,"N",49.426667,29.780000,0.000000,45.500000,0,"CH","FreeLocal","Standard",75.290000,2]

In [5]:
# create payload to send to deployed SPSS model for testing
fields=["Gender", "Status", "Children", "Est Income", "Car Owner", "Age", "LongDistance", "International", "Local", "Dropped", "Paymethod", "LocalBilltype", "LongDistanceBilltype", "Usage", "RatePlan"]

values= [array_of_values_to_be_scored, another_array_of_values_to_be_scored]


test_data = {
    "fields": fields,
    "values": values
}
print(type(test_data))
print(test_data)

<class 'dict'>
{'fields': ['Gender', 'Status', 'Children', 'Est Income', 'Car Owner', 'Age', 'LongDistance', 'International', 'Local', 'Dropped', 'Paymethod', 'LocalBilltype', 'LongDistanceBilltype', 'Usage', 'RatePlan'], 'values': [['F', 'S', 1, 38000.0, 'N', 24.393333, 23.56, 0.0, 206.08, 0, 'CC', 'Budget', 'Intnl_discount', 229.64, 3], ['M', 'M', 2, 29616.0, 'N', 49.426667, 29.78, 0.0, 45.5, 0, 'CH', 'FreeLocal', 'Standard', 75.29, 2]]}


In [6]:
# Score data against deployed SPSS model
payload = test_data
print(payload)
#result = client.deployments.score( function_deployment_endpoint_url, payload )
result = client.deployments.score( spss_scoring_url, payload )
if "error" in result:
    print( result["error"] )
else:
    print( result )

{'fields': ['Gender', 'Status', 'Children', 'Est Income', 'Car Owner', 'Age', 'LongDistance', 'International', 'Local', 'Dropped', 'Paymethod', 'LocalBilltype', 'LongDistanceBilltype', 'Usage', 'RatePlan'], 'values': [['F', 'S', 1, 38000.0, 'N', 24.393333, 23.56, 0.0, 206.08, 0, 'CC', 'Budget', 'Intnl_discount', 229.64, 3], ['M', 'M', 2, 29616.0, 'N', 49.426667, 29.78, 0.0, 45.5, 0, 'CH', 'FreeLocal', 'Standard', 75.29, 2]]}
{'fields': ['Gender', 'Status', 'Children', 'Est Income', 'Car Owner', 'Age', 'LongDistance', 'International', 'Local', 'Dropped', 'Paymethod', 'LocalBilltype', 'LongDistanceBilltype', 'Usage', 'RatePlan', '$C-CHURN', '$CC-CHURN'], 'values': [['F', 'S', 1, 38000.0, 'N', 24.393333, 23.56, 0.0, 206.08, 0, 'CC', 'Budget', 'Intnl_discount', 229.64, 3, 'T', 0.9915966386554622], ['M', 'M', 2, 29616.0, 'N', 49.426667, 29.78, 0.0, 45.5, 0, 'CH', 'FreeLocal', 'Standard', 75.29, 2, 'F', 0.964824120603015]]}


In [7]:
# list the WML repositories
client.repository.list()

------------------------------------  --------------------------------------------------------  ------------------------  -----------------  -----------------
GUID                                  NAME                                                      CREATED                   FRAMEWORK          TYPE
e18e3731-de87-4d70-a846-a194fdcb4ebe  TelusChurn_0603_hot_enc                                   2019-06-03T17:16:37.174Z  mllib              definition
5c77477a-95e4-4254-b6d1-a6a81dbf6b08  TelcoDemoSpark                                            2019-05-29T21:52:30.977Z  mllib              definition
33ba01fb-8988-4ef0-9571-be82df83cf84  TecloDemoSpark                                            2019-05-29T21:51:40.159Z  mllib              definition
d4e0d851-5935-4221-8406-51b804891069  TelusChurn_0520_hot_enc                                   2019-05-21T19:32:43.915Z  mllib              definition
18b27680-e8f6-44ce-808d-9d60355c525b  TelusChurn_0520_hot_enc                          

### WML Deployable Function
Create WML deployable function

In [8]:
ai_params = {}
def score_generator(params=ai_params):

    def score(payload):
        """AI function with model version.

        """
        
        from watson_machine_learning_client import WatsonMachineLearningAPIClient
        
        # Paste your credentials here 
        wml_credentials={
              "apikey": "*",
              "iam_apikey_description": "*",
              "iam_apikey_name": "*",
              "iam_role_crn": "*",
              "iam_serviceid_crn": "*",
              "instance_id": "*",
              "password": "*",
              "url": "*",
              "username": "*"
}


        
        client = WatsonMachineLearningAPIClient(wml_credentials)
        #scoring_url = params["scoring_url"]
        scoring_url='https://us-south.ml.cloud.ibm.com/v3/wml_instances/8654e5c8-9bda-4800-9b4d-31d69fa179c3/deployments/e4a73684-8150-4fcb-a2a0-6cbf229ba8e0/online'
        scores_model = client.deployments.score(scoring_url, payload)
        #print("scores_model")
        #print(scores_model)
        
        fields_model = payload["fields"] + ["probability"] + ["prediction"] + ["predictedLabel"]
        
        # The probability values and prediction values below are set for a binary classification model.
        
        values_model = []
        for i in range(len(scores_model["values"])):
            if scores_model["values"][i][-2] == "F":
                prediction = 0.0
                predicted_label = "F"
                if scores_model["values"][i][-1] is not None:
                    prob_list = [float(scores_model["values"][i][-1]), 1 - float(scores_model["values"][i][-1])]
            else:
                prediction = 1.0
                predicted_label = "T"
                if scores_model["values"][i][-1] is not None:
                    prob_list = [1 - float(scores_model["values"][i][-1]), float(scores_model["values"][i][-1])]
            
            values_model.append(payload["values"][i] + [prob_list] + [prediction] + [predicted_label])
        
        payload_model = {"fields": fields_model, "values": values_model}
        
        return payload_model

    return score

### Test the WML Deployable function locally, before deploying to WML.

In [9]:
#Define test data to send to the function

fields=["Gender", "Status", "Children", "Est Income", "Car Owner", "Age", "LongDistance", "International", "Local", "Dropped", "Paymethod", "LocalBilltype", "LongDistanceBilltype", "Usage", "RatePlan"]
values= [array_of_values_to_be_scored, another_array_of_values_to_be_scored]


test_data = {
    "fields": fields,
    "values": values
}
print(type(test_data))
print(test_data)

<class 'dict'>
{'fields': ['Gender', 'Status', 'Children', 'Est Income', 'Car Owner', 'Age', 'LongDistance', 'International', 'Local', 'Dropped', 'Paymethod', 'LocalBilltype', 'LongDistanceBilltype', 'Usage', 'RatePlan'], 'values': [['F', 'S', 1, 38000.0, 'N', 24.393333, 23.56, 0.0, 206.08, 0, 'CC', 'Budget', 'Intnl_discount', 229.64, 3], ['M', 'M', 2, 29616.0, 'N', 49.426667, 29.78, 0.0, 45.5, 0, 'CH', 'FreeLocal', 'Standard', 75.29, 2]]}


In [10]:
test_payload = test_data
# Pass the sample canvas data to the function as a test
#

func_result = score_generator()(test_payload)
print("func_result: ",func_result )

func_result:  {'fields': ['Gender', 'Status', 'Children', 'Est Income', 'Car Owner', 'Age', 'LongDistance', 'International', 'Local', 'Dropped', 'Paymethod', 'LocalBilltype', 'LongDistanceBilltype', 'Usage', 'RatePlan', 'probability', 'prediction', 'predictedLabel'], 'values': [['F', 'S', 1, 38000.0, 'N', 24.393333, 23.56, 0.0, 206.08, 0, 'CC', 'Budget', 'Intnl_discount', 229.64, 3, [0.008403361344537785, 0.9915966386554622], 1.0, 'T'], ['M', 'M', 2, 29616.0, 'N', 49.426667, 29.78, 0.0, 45.5, 0, 'CH', 'FreeLocal', 'Standard', 75.29, 2, [0.964824120603015, 0.035175879396984966], 0.0, 'F']]}


In [12]:
# Store the deployable function in your Watson Machine Learning repository
#
meta_data = { client.repository.FunctionMetaNames.NAME : 'CustomerChurn SPSS Function - scoring v4' }
function_details = client.repository.store_function( meta_props=meta_data, function=score_generator )

No RUNTIME_UID passed. Creating default runtime... SUCCESS

Successfully created default runtime with uid: 407fff90-b3f5-4bf6-91cc-f7b965c53960


In [13]:
# List the deployments in WML
# You should see a reference to the function you just deployed (check the Name field)
client.deployments.list()

------------------------------------  --------------------------------------------------------  ------  --------------  ------------------------  -----------------  -------------
GUID                                  NAME                                                      TYPE    STATE           CREATED                   FRAMEWORK          ARTIFACT TYPE
83e4d4cd-4c7f-48a7-bff7-85a532fb6a29  CustomerChurn SPSS function scoring branch deployment v3  online  DEPLOY_SUCCESS  2019-06-17T13:38:04.475Z  n/a                function
e4a73684-8150-4fcb-a2a0-6cbf229ba8e0  SPSS_c5_scoring_branch_depl                               online  DEPLOY_SUCCESS  2019-06-17T13:31:11.481Z  spss-modeler-18.1  model
36719e3e-48f8-4da8-9db1-9403f6ce33a8  SPSS_churn_c5                                             online  DEPLOY_SUCCESS  2019-06-12T13:18:40.303Z  spss-modeler-18.1  model
4e1090d0-e144-4e38-a1ff-81e328183519  Telus_hot_enc_0603                                        online  DEPLOY_SUCCESS  2019-0

In [41]:
client.deployments.delete('2262648b-1720-42d4-b5ef-7b70b9e3d0e4')

'SUCCESS'

In [42]:
# Deploy the stored function
#
function_id = function_details["metadata"]["guid"]
function_deployment_details = client.deployments.create( artifact_uid=function_id, name="CustomerChurn SPSS function scoring branch deployment v4" )




#######################################################################################

Synchronous deployment creation for uid: '0ad8c7e1-7a98-482e-9667-e893e2779eed' started

#######################################################################################


INITIALIZING
DEPLOY_IN_PROGRESS.
DEPLOY_SUCCESS


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='83e4d4cd-4c7f-48a7-bff7-85a532fb6a29'
------------------------------------------------------------------------------------------------




In [43]:
# List the deployments in WML
# You should see a reference to the function you just deployed (check the Name field)
client.deployments.list()

------------------------------------  --------------------------------------------------------  ------  --------------  ------------------------  -----------------  -------------
GUID                                  NAME                                                      TYPE    STATE           CREATED                   FRAMEWORK          ARTIFACT TYPE
83e4d4cd-4c7f-48a7-bff7-85a532fb6a29  CustomerChurn SPSS function scoring branch deployment v3  online  DEPLOY_SUCCESS  2019-06-17T13:38:04.475Z  n/a                function
e4a73684-8150-4fcb-a2a0-6cbf229ba8e0  SPSS_c5_scoring_branch_depl                               online  DEPLOY_SUCCESS  2019-06-17T13:31:11.481Z  spss-modeler-18.1  model
36719e3e-48f8-4da8-9db1-9403f6ce33a8  SPSS_churn_c5                                             online  DEPLOY_SUCCESS  2019-06-12T13:18:40.303Z  spss-modeler-18.1  model
4e1090d0-e144-4e38-a1ff-81e328183519  Telus_hot_enc_0603                                        online  DEPLOY_SUCCESS  2019-0

In [44]:
# Pull the deployment id from the previous cell that corresponds to the function you just deployed
deployment_id = '83e4d4cd-4c7f-48a7-bff7-85a532fb6a29'

In [45]:
function_deployment_details = client.deployments.get_details(deployment_id)

In [46]:
# Get the endpoint URL of the function deployment just created
#
function_deployment_endpoint_url = client.deployments.get_scoring_url( function_deployment_details )
function_deployment_endpoint_url

'https://us-south.ml.cloud.ibm.com/v3/wml_instances/8654e5c8-9bda-4800-9b4d-31d69fa179c3/deployments/83e4d4cd-4c7f-48a7-bff7-85a532fb6a29/online'

In [47]:
#Define test data to send to the function

fields=["Gender", "Status", "Children", "Est Income", "Car Owner", "Age", "LongDistance", "International", "Local", "Dropped", "Paymethod", "LocalBilltype", "LongDistanceBilltype", "Usage", "RatePlan"]
values= [array_of_values_to_be_scored, another_array_of_values_to_be_scored]


test_data2 = {
    "fields": fields,
    "values": values
}
print(type(test_data2))
print(test_data2)

<class 'dict'>
{'values': [['F', 'S', 1, 38000.0, 'N', 24.393333, 23.56, 0.0, 206.08, 0, 'CC', 'Budget', 'Intnl_discount', 229.64, 3], ['M', 'M', 2, 29616.0, 'N', 49.426667, 29.78, 0.0, 45.5, 0, 'CH', 'FreeLocal', 'Standard', 75.29, 2]], 'fields': ['Gender', 'Status', 'Children', 'Est Income', 'Car Owner', 'Age', 'LongDistance', 'International', 'Local', 'Dropped', 'Paymethod', 'LocalBilltype', 'LongDistanceBilltype', 'Usage', 'RatePlan']}


In [48]:
payload = test_data2
print(payload)
result = client.deployments.score( function_deployment_endpoint_url, payload )
#result = client.deployments.score( spss_scoring_url, payload )
if "error" in result:
    print( result["error"] )
else:
    print( result )

{'values': [['F', 'S', 1, 38000.0, 'N', 24.393333, 23.56, 0.0, 206.08, 0, 'CC', 'Budget', 'Intnl_discount', 229.64, 3], ['M', 'M', 2, 29616.0, 'N', 49.426667, 29.78, 0.0, 45.5, 0, 'CH', 'FreeLocal', 'Standard', 75.29, 2]], 'fields': ['Gender', 'Status', 'Children', 'Est Income', 'Car Owner', 'Age', 'LongDistance', 'International', 'Local', 'Dropped', 'Paymethod', 'LocalBilltype', 'LongDistanceBilltype', 'Usage', 'RatePlan']}
{'values': [['F', 'S', 1, 38000.0, 'N', 24.393333, 23.56, 0.0, 206.08, 0, 'CC', 'Budget', 'Intnl_discount', 229.64, 3, [0.008403361344537785, 0.9915966386554622], 1.0, 'T'], ['M', 'M', 2, 29616.0, 'N', 49.426667, 29.78, 0.0, 45.5, 0, 'CH', 'FreeLocal', 'Standard', 75.29, 2, [0.964824120603015, 0.035175879396984966], 0.0, 'F']], 'fields': ['Gender', 'Status', 'Children', 'Est Income', 'Car Owner', 'Age', 'LongDistance', 'International', 'Local', 'Dropped', 'Paymethod', 'LocalBilltype', 'LongDistanceBilltype', 'Usage', 'RatePlan', 'probability', 'prediction', 'predic

### As function is deployed in WML, use Openscale UI to set it up for monitoring